In [7]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting 환경 설정
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# 데이터를 구분하는 문자를 지정하는 옵션
vod8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_VOD_2308.csv', sep='\t', encoding='cp949')
vod9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_VOD_2309.csv', sep='\t', encoding='cp949')
# channel8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨3차_채널PGM_2308월.csv', sep='\t', encoding='cp949')
# channel9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨3차_채널PGM_2309월.csv', encoding='cp949')
con8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_콘텐츠_2308.csv', sep='\t', encoding='cp949')
con9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_콘텐츠_2309.csv', sep='\t', encoding='cp949')

In [4]:
v8 = vod8.copy()
v9 = vod9.copy()
c8 = con8.copy()
c9 = con9.copy()

In [5]:
con9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14387 entries, 0 to 14386
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           14387 non-null  int64 
 1   series_nm       6458 non-null   object
 2   super_asset_nm  14387 non-null  object
 3   ct_cl           14387 non-null  object
 4   genre_of_ct_cl  14387 non-null  object
 5   SMRY            14387 non-null  object
 6   ACTR_DISP       14387 non-null  object
 7   disp_rtm        13260 non-null  object
 8   log_dt          14387 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1011.7+ KB


# content 전처리

In [10]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
con89 = pd.concat([con8, con9], ignore_index=True)
con89.tail(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
30063,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121654


In [84]:
# 예고가 위치한 컬럼 확인
con89['super_asset_nm'].str.contains("예고").value_counts()

False    30059
True         5
Name: super_asset_nm, dtype: int64

In [62]:
from pandas._libs.tslibs.offsets import Hour
import re
def preprocessing_con(df):
  # 특수문자 정리
  ## 예고편 제거
  df = df[~df['super_asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

  # series_nm 괄호와 그 안의 내용 제거
  df['series_nm'] = df['series_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace('-', ' ')
  df['series_nm'] = df['series_nm'].str.rstrip('.')

  # super_asset_nm 변경
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace('-', ' ')
  df['super_asset_nm'] = df['super_asset_nm'].str.rstrip('.')

  # 회, 회. 제거
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회$', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['series_nm'] = df['series_nm'].str.replace(r'\d+화$', '', regex = True)

  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회$', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['super_asset_nm'] = df['super_asset_nm'].str.replace(r'\d+화$', '', regex = True)

  # 'series_nm'이 null인 경우, 'super_asset_nm'의 값으로 대체
  df['series_nm'].fillna(df['super_asset_nm'], inplace=True)

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0

    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  return df

In [63]:
con_89 = preprocessing_con(con89)
con_89.head(3)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155
1,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071215
2,66056000,라바 키즈동요 Part4,라바 키즈동요 Part4,키즈,기타,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",2,20230808085440


In [65]:
# 확인
con_89[con_89["series_nm"].isnull()]

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt


In [68]:
# 새로운 변수 추가
new_column_name  = 'summary'
con_89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
con_89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
con_89["category"] = pd.Series(dtype='object')
con_89.head(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,summary,director,category
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,1,20230813071155,NaN,NaN,NaN


In [73]:
new_order = ["series_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm", "SMRY"]
con = con_89[new_order]
con.tail(1)

,series_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


In [76]:
# 이름 변경
con.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
con.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,12,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


# vod 전처리

In [66]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
vod89 = pd.concat([vod8, vod9], ignore_index=True)
vod89.tail(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
10656,60169000,놀자!,키즈,학습,407,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...,-,0:07,20230914210135


In [69]:
# asset_nm 괄호와 괄호 안의 데이터 제거
def preprocessing_vod(df):
  # 특수문자 정리
  ## 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

  # series_nm 괄호와 그 안의 내용 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace('-', ' ')
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')

  # 회, 회. 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+회$', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\d+화$', '', regex = True)

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0

    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  return df

In [71]:
vod_89 = preprocessing_vod(vod89)
vod_89.head(3)

<ipython-input-69-25d68c75a793>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
<ipython-input-69-25d68c75a793>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
<ipython-input-69-25d68c75a793>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다,TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727


In [72]:
new_column_name  = 'summary'
vod_89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
vod_89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
vod_89["category"] = pd.Series(dtype='object')
vod_89.head(1)

<ipython-input-72-be88defb1a4f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_89["summary"] = pd.Series(dtype='object')
<ipython-input-72-be88defb1a4f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_89["director"] = pd.Series(dtype='object')
<ipython-input-72-be88defb1a4f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,summary,director,category
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507,NaN,NaN,NaN


In [75]:
# 순서 변경
new_order = ["asset_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm","SMRY"]
vod = vod_89[new_order]
vod.tail(1)

,asset_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,7,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [77]:
vod.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
vod.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,7,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [78]:
vod_data = pd.concat([con, vod], ignore_index=True)
vodd = vod_data.copy()
vod_data.head(5)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,1,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,1,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
2,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,2,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
3,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,2,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
4,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,NaN,11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...


### 중복 제거

In [89]:
# 'name'과 'BigCategory'를 기준으로 중복된 값 삭제
vod_data = vod_data.drop_duplicates(subset=['name', 'BigCategory'])
vod_data = vod_data.reset_index(drop=True)
vod_data.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
5147,놀자!,NaN,-,NaN,키즈,학습,키즈,7,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [91]:
# Category 구분
vod_data['Category'] = np.where(vod_data['BigCategory'] == '영화', '영화', np.where(vod_data['BigCategory'] == '키즈', '키즈', 'TV프로그램'))
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,1,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,2,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
2,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...
3,엉뚱남매 요리조리쇼,NaN,"기원,예현,홍윤화",NaN,키즈,기타,키즈,11,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...
4,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,2,CLEAN UP SONG. Clean up Clean up everybody cle...
...,...,...,...,...,...,...,...,...,...
5143,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,8,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
5144,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,4,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
5145,간질간질,NaN,-,NaN,키즈,학습,키즈,6,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
5146,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,1,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


### 확인

In [81]:
vod_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40487 entries, 0 to 40486
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           40487 non-null  object
 1   Summary        0 non-null      object
 2   actors         40483 non-null  object
 3   director       0 non-null      object
 4   BigCategory    40487 non-null  object
 5   SmallCategory  40487 non-null  object
 6   Category       40487 non-null  object
 7   runningtime    40487 non-null  int64 
 8   SMRY           40483 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.8+ MB


In [82]:
# '-', '[]', '.'이 들어간 값 확인
# filtered_rows = vod_data[vod_data['name'].str.contains('\-')]
# filtered_rows = vod_data[vod_data['name'].str.contains('\[\]')]
end = vod_data['name'].str.endswith('.')
end_giho = vod_data[end]
end_giho

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY


In [92]:
vod_all.to_csv('vod_all.csv', index=False, encoding='cp949')

NameError: ignored